# Notebook de nettoyage - Adrian Rodriguez - Projet 3 parcours Ingénieur Machine Learning

---
# 0. Contexte


Vous êtes consultant pour Olist, une solution de vente sur les marketplaces en ligne.

Olist souhaite que vous fournissiez à ses équipes d'e-commerce une segmentation des clients qu’elles pourront utiliser au quotidien pour leurs campagnes de communication.

Votre objectif est de comprendre les différents types d’utilisateurs grâce à leur comportement et à leurs données personnelles.

Vous devrez fournir à l’équipe marketing une description actionable de votre segmentation et de sa logique sous-jacente pour une utilisation optimale, ainsi qu’une proposition de contrat de maintenance basée sur une analyse de la stabilité des segments au cours du temps.

### Les données
Pour cette mission, Olist vous fournit une base de données anonymisée comportant des informations sur l’historique de commandes, les produits achetés, les commentaires de satisfaction, et la localisation des clients depuis janvier 2017.

### Votre mission
Votre mission est d’aider les équipes d’Olist à comprendre les différents types d'utilisateurs. Vous utiliserez donc des méthodes non supervisées pour regrouper ensemble des clients de profils similaires. Ces catégories pourront être utilisées par l’équipe marketing pour mieux communiquer.

Pour des raisons de confidentialité, Olist ne fournit pas beaucoup de données, à vous de fouiller dans celles dont vous disposez et de créer les meilleures features pour les exploiter.

Enfin, votre client, Olist, a spécifié sa demande ainsi :

- La segmentation proposée doit être exploitable et facile d’utilisation pour l’équipe marketing.
- Vous évaluerez la fréquence à laquelle la segmentation doit être mise à jour, afin de pouvoir effectuer un devis de contrat de maintenance.
- Le code fourni doit respecter la convention PEP8, pour être utilisable par Olist.

# 1. Importation et exploration préliminaire

In [28]:
import pandas as pd
from pandas_profiling import ProfileReport
import glob
import os

pd.options.display.max_columns = None

# Pour vérifier le respect de la convention PEP8
# Installation conda non diponible. Utiliser pip
%load_ext pycodestyle_magic
%pycodestyle_on

The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


In [22]:
df_dict = {}
for file in glob.glob('src/olist/*csv'):
    df_name = file.replace('src/olist\\', 'df_').replace('olist_', '')
    df_name = df_name.replace('_dataset.csv', '').replace('.csv', '')
    print(df_name)
    df_dict[df_name] = pd.read_csv(file)

df_customers
df_geolocation
df_orders
df_order_items
df_order_payments
df_order_reviews
df_products
df_sellers
df_product_category_name_translation


In [17]:
for name, df in df_dict.items():
    display(name)
    display(df.info())
    display(df.head())

'src/olist\\customers'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


None

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


'src/olist\\geolocation'

...

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


'src/olist\\product_category_name_translation'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 2 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   product_category_name          71 non-null     object
 1   product_category_name_english  71 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB


None

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


<img src="src/relation.png"/>

In [23]:
merge_list = [df_dict['df_orders'],
              df_dict['df_order_items'],
              df_dict['df_order_reviews'],
              df_dict['df_order_payments'],
              df_dict['df_products'],
              df_dict['df_product_category_name_translation']]

df_raw = df_dict['df_customers']
for df in merge_list:
    df_raw = pd.merge(df_raw, df)

df_raw.shape

(116581, 37)

In [24]:
df_raw.dtypes.to_csv('src/columns.csv')

In [10]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116581 entries, 0 to 116580
Data columns (total 37 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   customer_id                    116581 non-null  object 
 1   customer_unique_id             116581 non-null  object 
 2   customer_zip_code_prefix       116581 non-null  int64  
 3   customer_city                  116581 non-null  object 
 4   customer_state                 116581 non-null  object 
 5   order_id                       116581 non-null  object 
 6   order_status                   116581 non-null  object 
 7   order_purchase_timestamp       116581 non-null  object 
 8   order_approved_at              116567 non-null  object 
 9   order_delivered_carrier_date   115368 non-null  object 
 10  order_delivered_customer_date  114066 non-null  object 
 11  order_estimated_delivery_date  116581 non-null  object 
 12  order_item_id                 

In [27]:
df_raw

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,payment_sequential,payment_type,payment_installments,payment_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05 00:00:00,1,a9516a079e37a9c9c36b9b78b10169e8,7c67e1448b00f6e969d365cea6b010ab,2017-05-22 15:22:12,124.99,21.88,88b8b52d46df026a9d1ad2136a59b30b,4,NaN,NaN,2017-05-26 00:00:00,2017-05-30 22:34:40,1,credit_card,2,146.87,moveis_escritorio,41.0,1141.0,1.0,8683.0,54.0,64.0,31.0,office_furniture
1,8912fc0c3bbf1e2fbf35819e21706718,9eae34bbd3a474ec5d07949ca7de67c0,68030,santarem,PA,c1d2b34febe9cd269e378117d6681172,delivered,2017-11-09 00:50:13,2017-11-10 00:47:48,2017-11-22 01:43:37,2017-11-28 00:09:50,2017-12-19 00:00:00,1,a9516a079e37a9c9c36b9b78b10169e8,7c67e1448b00f6e969d365cea6b010ab,2017-11-23 00:47:18,112.99,24.90,7fc63200f12eebb5f387856afdd63db8,1,NaN,"GOSTARIA DE UMA SOLUÇÃO, ESTOU PRECISANDO MUIT...",2017-11-29 00:00:00,2017-12-01 19:41:59,1,credit_card,1,275.79,moveis_escritorio,41.0,1141.0,1.0,8683.0,54.0,64.0,31.0,office_furniture
2,8912fc0c3bbf1e2fbf35819e21706718,9eae34bbd3a474ec5d07949ca7de67c0,68030,santarem,PA,c1d2b34febe9cd269e378117d6681172,delivered,2017-11-09 00:50:13,2017-11-10 00:47:48,2017-11-22 01:43:37,2017-11-28 00:09:50,2017-12-19 00:00:00,2,a9516a079e37a9c9c36b9b78b10169e8,7c67e1448b00f6e969d365cea6b010ab,2017-11-23 00:47:18,112.99,24.90,7fc63200f12eebb5f387856afdd63db8,1,NaN,"GOSTARIA DE UMA SOLUÇÃO, ESTOU PRECISANDO MUIT...",2017-11-29 00:00:00,2017-12-01 19:41:59,1,credit_card,1,275.79,moveis_escritorio,41.0,1141.0,1.0,8683.0,54.0,64.0,31.0,office_furniture
3,f0ac8e5a239118859b1734e1087cbb1f,3c799d181c34d51f6d44bbbc563024db,92480,nova santa rita,RS,b1a5d5365d330d10485e0203d54ab9e8,delivered,2017-05-07 20:11:26,2017-05-08 22:22:56,2017-05-19 20:16:31,2017-05-26 09:54:04,2017-06-12 00:00:00,1,a9516a079e37a9c9c36b9b78b10169e8,7c67e1448b00f6e969d365cea6b010ab,2017-05-22 22:22:56,124.99,15.62,251191809e37c1cffc16865947c18a4d,3,NaN,"Produto compatível com seu valor, muito bonito...",2017-05-27 00:00:00,2017-05-28 02:43:16,1,credit_card,7,140.61,moveis_escritorio,41.0,1141.0,1.0,8683.0,54.0,64.0,31.0,office_furniture
4,6bc8d08963a135220ed6c6d098831f84,23397e992b09769faf5e66f9e171a241,25931,mage,RJ,2e604b3614664aa66867856dba7e61b7,delivered,2018-02-03 19:45:40,2018-02-04 22:29:19,2018-02-19 18:21:47,2018-02-28 21:09:00,2018-03-22 00:00:00,1,a9516a079e37a9c9c36b9b78b10169e8,7c67e1448b00f6e969d365cea6b010ab,2018-02-18 21:29:19,106.99,30.59,f7123bac5b91a0e2e38d8b41fd1206f4,4,NaN,Entregou antes do prazo,2018-03-01 00:00:00,2018-03-02 11:11:24,1,credit_card,10,137.58,moveis_escritorio,41.0,1141.0,1.0,8683.0,54.0,64.0,31.0,office_furniture
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116576,8325d6e7af2f2bfb58cf040729b232dd,a1d53377ab83871b0ad483b03ac69d6c,3570,sao paulo,SP,b99217b1fcf2fdeace7b8d869b76cb91,delivered,2018-03-07 17:42:06,2018-03-07 17:55:36,2018-03-08 20:09:40,2018-03-15 22:04:49,2018-04-02 00:00:00,1,57bdf3098169cccdb62221bd3e089cbd,d98eec89afa3380e14463da2aabaea72,2018-03-13 17:55:36,39.99,10.15,048757148581cbd2c92bc96ca5d73211,5,NaN,NaN,2018-03-16 00:00:00,2018-03-19 09:12:20,1,credit_card,3,110.44,fashion_roupa_i

In [39]:
dim = pd.read_excel('dictionnaire.xlsx', sheet_name='variable')
columns = dim.variables[dim.model == 'yes'].to_list()

In [41]:
df = df_raw[columns]
df

,customer_id,price,freight_value,review_score,payment_sequential,payment_type,payment_value,product_name_lenght,product_description_lenght,product_photos_qty,product_category_name_english
0,06b8999e2fba1a1fbc88172c00ba8bc7,124.99,21.88,4,1,credit_card,146.87,41.0,1141.0,1.0,office_furniture
1,8912fc0c3bbf1e2fbf35819e21706718,112.99,24.90,1,1,credit_card,275.79,41.0,1141.0,1.0,office_furniture
2,8912fc0c3bbf1e2fbf35819e21706718,112.99,24.90,1,1,credit_card,275.79,41.0,1141.0,1.0,office_furniture
3,f0ac8e5a239118859b1734e1087cbb1f,124.99,15.62,3,1,credit_card,140.61,41.0,1141.0,1.0,office_furniture
4,6bc8d08963a135220ed6c6d098831f84,106.99,30.59,4,1,credit_card,137.58,41.0,1141.0,1.0,office_furniture
...,...,...,...,...,...,...,...,...,...,...,...
116576,8325d6e7af2f2bfb58cf040729b232dd,39.99,10.15,5,1,credit_card,110.44,38.0,168.0,1.0,fashion_childrens_clothes
116577,85485f38b20ed1e287120d30cf39ee7d,39.99,14.10,5,1,credit_card,54.09,38.0,168.0,1.0,fashion_childrens_clothes
116578,14acb30bbc064408dcb8cdd3c92b0c97,110.00,14.52,5,1,credit_card,124.52,52.0,494.0,4.0,fashion_childrens_clothes
116579,363d3a9b2ec5c5426608688ca033292d,183.29,25.77,4,1,credit_card,209.06,55.0,506.0,1.0,security_and_services


In [47]:
category = df.groupby('product_category_name_english')['customer_id'].count()
category.sort_values(ascending=False).head(20)

product_category_name_english
bed_bath_table              11990
health_beauty               10030
sports_leisure               9005
furniture_decor              8833
computers_accessories        8151
housewares                   7380
watches_gifts                6213
telephony                    4726
garden_tools                 4590
auto                         4400
toys                         4281
cool_stuff                   3999
perfumery                    3589
baby                         3204
electronics                  2847
stationery                   2625
fashion_bags_accessories     2170
pet_shop                     2030
office_furniture             1788
consoles_games               1192
Name: customer_id, dtype: int64